# This notebook is used for modelling between the livability of melbourne and rental affordability

For the sake of simplicty, this notebook contains the following
- Livability scores of melbourne
- Rental affordability of greater Melbourne

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.api import anova_lm
from scipy.stats import pearsonr
import statsmodels.api as sm
import plotly
import plotly.express as px
import plotly.graph_objs as go
from elasticsearch import Elasticsearch

## Data processing

The below data will be used for this model:

### Melbourne livability score

- This dataset contains multiple liveability related scores for multiple regions

### Predictor variable: Greater Melbourne rental affordability index

- This dataset contains information about the rental affordability 

## Initial data analysis

First, load the data from ESDB

In [ ]:
INDEX_NAME = "liveability-index"

es_client = Elasticsearch (
        'https://localhost:9200',
        verify_certs= False,
        ssl_show_warn= False,
        basic_auth=('elastic', 'elastic')
    )
es_client.indices.refresh(index=INDEX_NAME)
result = es_client.search(index=INDEX_NAME, body = {
'size' : 1000,
'query': {
    'match_all' : {}
}
})


es_data = []
for doc in result['hits']['hits']:
    es_data.append(doc['_source'])
liveability = pd.DataFrame(es_data)

In [ ]:
INDEX_NAME = "rental_affordability"

es_client = Elasticsearch (
        'https://localhost:9200',
        verify_certs= False,
        ssl_show_warn= False,
        basic_auth=('elastic', 'elastic')
    )
es_client.indices.refresh(index=INDEX_NAME)
result = es_client.search(index=INDEX_NAME, body = {
'size' : 1000,
'query': {
    'match_all' : {}
}
})


es_data = []
for doc in result['hits']['hits']:
    es_data.append(doc['_source'])
rental_affordability = pd.DataFrame(es_data)

In [3]:
# Load data

liveability = pd.read_csv("../../data/predictor/Melbourne Liveability Indicators.csv")
rental_affordability = pd.read_csv("../../data/response/rental_affordability.csv")
rental_affordability.columns = rental_affordability.columns.str.replace(' ', '')

In [4]:
rental_affordability.columns = rental_affordability.columns.str.replace(' ', '')
liveability.columns.sort_values()

Index(['denominator', 'id', 'indicator', 'numerator', 'period', 'sources',
       'topic', 'type', 'value', 'value_type'],
      dtype='object')

In [5]:
set(liveability['period'])

{'2006',
 '2011',
 '2012',
 '2013',
 '2013/14',
 '2014',
 '2014 (June Qtr)',
 '2014 (at 30 June)',
 '2014/15',
 '2015',
 '2015 (June Qtr)',
 '2015 (at 30 June)',
 '2015/16',
 '2016',
 '2016 (June Qtr)',
 '2016 (at 30 June)',
 '2016/17',
 '2017',
 '2017 (June Qtr)',
 '2017 (at 30 June)',
 '2018',
 '2018 (June Qtr)',
 '2018 (at 30 June)',
 '2019',
 'FY 2013/14',
 'FY 2014/15',
 'FY 2015/16',
 'FY 2016/17',
 'FY 2017/18',
 'FY 202013/14',
 'FY 202014/15',
 'FY 202015/16'}

For the liveability dataset, we have information from 2006, 2011~2019. Now we proceed to look at rental_affordability

In [6]:
rental_affordability

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
0,3000,98.33916,98.33916,109.61540,93.91771,VIC,144.2308,109.61540,137.0192,100.62610,...,97.23423,102.56410,103.71650,107.33450,96.15385,107.33450,102.79250,102.56410,102.56410,114.04290
1,3003,97.23423,100.62610,107.46610,88.75740,VIC,144.2308,109.61540,121.7949,81.64006,...,94.06355,103.71650,100.33440,107.33450,90.14423,102.56410,102.56410,113.96010,99.25558,119.60600
2,3004,89.21491,88.30455,103.41070,84.13462,VIC,130.3781,105.39940,111.8524,99.46950,...,86.53846,96.15385,94.19152,98.19967,90.14423,97.16599,96.15385,102.56410,97.16599,102.16350
3,3005,NaN,NaN,NaN,NaN,VIC,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3006,81.64006,78.67133,99.65035,77.66272,VIC,144.2308,99.65035,121.7949,83.21006,...,81.64006,87.91209,88.75740,88.75740,81.64006,90.49774,87.08273,90.49774,90.49774,98.07692
5,3008,77.26648,83.21006,95.31773,74.78632,VIC,135.7466,97.87088,105.3994,79.39308,...,82.41758,90.49774,92.30769,89.61912,88.30455,92.30769,90.49774,93.24009,94.19152,96.15385


In [7]:
rental_affordability.columns.sort_values()

Index(['city', 'geography_name', 'rai_cityadjusted_total_2011_q1',
       'rai_cityadjusted_total_2011_q2', 'rai_cityadjusted_total_2011_q3',
       'rai_cityadjusted_total_2011_q4', 'rai_cityadjusted_total_2012_q1',
       'rai_cityadjusted_total_2012_q2', 'rai_cityadjusted_total_2012_q3',
       'rai_cityadjusted_total_2012_q4', 'rai_cityadjusted_total_2013_q1',
       'rai_cityadjusted_total_2013_q2', 'rai_cityadjusted_total_2013_q3',
       'rai_cityadjusted_total_2013_q4', 'rai_cityadjusted_total_2014_q1',
       'rai_cityadjusted_total_2014_q2', 'rai_cityadjusted_total_2014_q3',
       'rai_cityadjusted_total_2014_q4', 'rai_cityadjusted_total_2015_q1',
       'rai_cityadjusted_total_2015_q2', 'rai_cityadjusted_total_2015_q3',
       'rai_cityadjusted_total_2015_q4', 'rai_cityadjusted_total_2016_q1',
       'rai_cityadjusted_total_2016_q2', 'rai_cityadjusted_total_2016_q3',
       'rai_cityadjusted_total_2016_q4', 'rai_cityadjusted_total_2017_q1',
       'rai_cityadjusted_total_20

From a quick look , it looks like geography_name=3005 seems to only hold NaN values, and thus can get rid of it

In [8]:
rental_affordability[rental_affordability['geography_name']==3005]

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
3,3005,NaN,NaN,NaN,NaN,VIC,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
rental_affordability[['unique_id', 'city', 'geography_name', 'state']]

,unique_id,city,geography_name,state
0,1310,Greater Melbourne,3000,VIC
1,1312,Greater Melbourne,3003,VIC
2,1313,Greater Melbourne,3004,VIC
3,1314,Greater Melbourne,3005,VIC
4,1315,Greater Melbourne,3006,VIC
5,1316,Greater Melbourne,3008,VIC


From the above information, we can remove all columns but geography_name (which seems to be postcode), and also remove where geography_name=3005

In [10]:
rental_affordability = rental_affordability.drop(3)
rental_affordability = rental_affordability.reset_index()
rental_affordability.drop(columns=rental_affordability.columns[0], axis=1, inplace=True)

## Dataset cleanup
Now that we have the appropriate information, we finalise the dataset so that we can use for modelliing

In [11]:
liveability

,type,topic,id,indicator,period,numerator,denominator,value,value_type,sources
0,Liveability,Economy,ECO_1,City's unemployment rate,2015 (June Qtr),"3,942","84,942",4.64,Percentage,Australian Government Department of Employment...
1,Liveability,Economy,ECO_2,Assessed value of commercial and industrial pr...,2015 (at 30 June),"48,331,069,777.00","101,671,404,180.00",47.54,Percentage,"City of Melbourne, Property Services, 2015-2018"
2,Liveability,Economy,ECO_3,Percentage of city population living in poverty,2015,"13,177","136,872",9.63,Percentage,"Geografia, Relative Poverty and Employment Mea..."
3,Liveability,Economy,ECO_4,Percentage of persons (city population) in ful...,2017 (June Qtr),"68,722","159,141",43.18,Percentage,"Geografia, Relative Poverty and Employment Mea..."
4,Liveability,Economy,ECO_4,Percentage of persons (city population) in ful...,2014 (June Qtr),"55,359","127,975",43.26,Percentage,"Geografia, Relative Poverty and Employment Mea..."
...,...,...,...,...,...,...,...,...,...,...
314,Social,Participation in activities,ACT_7,Percentage of residents who have participated ...,2019,NaN,"1,259",49.30,Percentage,City of Melbourne Social Indicators Survey 2019
315,Social,"Culture, diversity and safety",CULT_4,Percentage of residents that reported feeling ...,2019,NaN,"1,235",57.40,Percentage,City of Melbourne Social Indicators Survey 2019
316,Social,"Culture, diversity and safety",CULT_5,Percentage of residents that reported feeling ...,2019,NaN,"1,197",86.00,Percentage,City of Melbourne Social Indicators Survey 2019
317,Social,"Culture, diversity and safety",CULT_5,Percentage of residents that reported feeling ...,2018,NaN,"1,188",86.50,Percentage,City of Melbourne Social Indicators Survey 2018


In [12]:
rental_affordability

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
0,3000,98.33916,98.33916,109.61540,93.91771,VIC,144.2308,109.61540,137.0192,100.62610,...,97.23423,102.56410,103.71650,107.33450,96.15385,107.33450,102.79250,102.56410,102.56410,114.04290
1,3003,97.23423,100.62610,107.46610,88.75740,VIC,144.2308,109.61540,121.7949,81.64006,...,94.06355,103.71650,100.33440,107.33450,90.14423,102.56410,102.56410,113.96010,99.25558,119.60600
2,3004,89.21491,88.30455,103.41070,84.13462,VIC,130.3781,105.39940,111.8524,99.46950,...,86.53846,96.15385,94.19152,98.19967,90.14423,97.16599,96.15385,102.56410,97.16599,102.16350
3,3006,81.64006,78.67133,99.65035,77.66272,VIC,144.2308,99.65035,121.7949,83.21006,...,81.64006,87.91209,88.75740,88.75740,81.64006,90.49774,87.08273,90.49774,90.49774,98.07692
4,3008,77.26648,83.21006,95.31773,74.78632,VIC,135.7466,97.87088,105.3994,79.39308,...,82.41758,90.49774,92.30769,89.61912,88.30455,92.30769,90.49774,93.24009,94.19152,96.15385


Firstly, we need to be able to condense both of the datasets into year so that we can model easier. For the liveability score, we'll combine numerator + denominator for a whole year, then calculate a new percentage for said year. Further, we only want percentages for our dataset since counts will not give us any relevant predictions, as we do not need to create a model but rather see some correlation between the 2 datasets

In [13]:
# Get the subset of the dataframe with the data we want

liveability = liveability[liveability['type'] == 'Liveability']
liveability = liveability[liveability['value_type'] == 'Percentage']
liveability['period'] = liveability['period'].str.extract(r'(\d{4})')
set(liveability['topic'])

{'Economy',
 'Finance',
 'Governance',
 'Profile - housing, government and economy',
 'Profile - people',
 'Transportation'}

In [14]:
# Then remove all NaN values

liveability = liveability.dropna(subset=['numerator', 'denominator'])

In [15]:
# Turn all numerical columns into int for calculation

liveability['numerator'] = liveability['numerator'].astype(str).str.replace(',', '').astype(float).astype(int)
liveability['denominator'] = liveability['denominator'].astype(str).str.replace(',', '').astype(float).astype(int)

In [16]:
liveability[liveability['indicator'] == 'Percentage of country GDP']

,type,topic,id,indicator,period,numerator,denominator,value,value_type,sources
46,Liveability,"Profile - housing, government and economy",PROH_8,Percentage of country GDP,2016,1323421072479,73043232000,5.52,Percentage,"1. The World Bank data, GDP (current US$) for ..."
47,Liveability,"Profile - housing, government and economy",PROH_8,Percentage of country GDP,2015,1208039015868,68408312000,5.66,Percentage,"1. The World Bank data, GDP (current US$) for ..."
48,Liveability,"Profile - housing, government and economy",PROH_8,Percentage of country GDP,2013,1464955475993,83367000000,5.69,Percentage,"1. The World Bank data, GDP (current US$) for ..."
247,Liveability,"Profile - housing, government and economy",PROH_8,Percentage of country GDP,2014,1349034029453,69580800000,5.16,Percentage,"1. The World Bank data, GDP (current US$) for ..."


After a qucik look, for some reason the numerator & denominator for all GDP related rows are swapped. Further, even if we did swap the columns around for appropriate values, the numbers are much greater than the other values used for calculation, thus adding these values in will skew our data so that the majority of the percentage calculation would come from these 2 rows. Thus, opt to remove these rows

In [17]:
liveability = liveability[liveability['indicator'] != 'Percentage of country GDP']

In [18]:
liveability[liveability['period'] == '2014']

,type,topic,id,indicator,period,numerator,denominator,value,value_type,sources
4,Liveability,Economy,ECO_4,Percentage of persons (city population) in ful...,2014,55359,127975,43.26,Percentage,"Geografia, Relative Poverty and Employment Mea..."
5,Liveability,Economy,ECO_5,Youth unemployment rate,2014,1367,13048,10.48,Percentage,"Geografia, Relative Poverty and Employment Mea..."
12,Liveability,Finance,FIN_1,Debt service ratio (debt service expenditure a...,2014,0,378466000,0.00,Percentage,1-3. City of Melbourne Annual Report 2017-18; ...
15,Liveability,Finance,FIN_2,Capital spending as a percentage of total expe...,2014,82078000,457698000,17.93,Percentage,1-3. City of Melbourne Annual Report 2017-18; ...
18,Liveability,Finance,FIN_4,Tax collected as a percentage of tax billed,2014,228981000,226920000,100.91,Percentage,1-3. City of Melbourne Annual Report 2017-18; ...
30,Liveability,Governance,GOV_3,Percentage of women employed in the city gover...,2014,1019,2631,38.73,Percentage,1-3. City of Melbourne Annual Report 2017-18; ...
51,Liveability,Profile - people,PROP_10,Percentage of population that are adults (25-6...,2014,74084,127975,57.89,Percentage,"1. Australian Bureau of Statistics, 3235.0 Reg..."
143,Liveability,Profile - people,PROP_12,Population dependency ratio (people aged under...,2014,13,86,15.52,Percentage,"1. Australian Bureau of Statistics, 3235.0 Reg..."
146,Liveability,Profile - people,PROP_5,Percentage of population that are seniors (65+),2014,8413,127975,6.57,Percentage,"Australian Bureau of Statistics, Census of Pop..."
202,Liveability,Economy,ECO_3,Percentage of city population living in poverty,2014,11556,127975,9.03,Percentage,"Geografia, Relative Poverty and Employment Mea..."


In [19]:
liveability_clean = liveability[['period', 'numerator', 'denominator']].groupby('period').sum().reset_index()
liveability_clean['percentage'] = ((liveability_clean['numerator'] * 100)/liveability_clean['denominator']).round(2)
liveability_clean.rename(columns={'period': 'year'}, inplace=True)

In [20]:
liveability_final = liveability_clean[['year', 'percentage']]

Note that for this liveability scoring dataset, it doesn't make much sense to add the differing factors to simply calculate a new 'percentage' per year. However, since the goal of this analysis is to see if there a correlation between the 2 datasets, so we ignore this for the sake of simplicity and understanding :)

Now we clean up the dataset for rental affordability

In [21]:
rental_affordability

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
0,3000,98.33916,98.33916,109.61540,93.91771,VIC,144.2308,109.61540,137.0192,100.62610,...,97.23423,102.56410,103.71650,107.33450,96.15385,107.33450,102.79250,102.56410,102.56410,114.04290
1,3003,97.23423,100.62610,107.46610,88.75740,VIC,144.2308,109.61540,121.7949,81.64006,...,94.06355,103.71650,100.33440,107.33450,90.14423,102.56410,102.56410,113.96010,99.25558,119.60600
2,3004,89.21491,88.30455,103.41070,84.13462,VIC,130.3781,105.39940,111.8524,99.46950,...,86.53846,96.15385,94.19152,98.19967,90.14423,97.16599,96.15385,102.56410,97.16599,102.16350
3,3006,81.64006,78.67133,99.65035,77.66272,VIC,144.2308,99.65035,121.7949,83.21006,...,81.64006,87.91209,88.75740,88.75740,81.64006,90.49774,87.08273,90.49774,90.49774,98.07692
4,3008,77.26648,83.21006,95.31773,74.78632,VIC,135.7466,97.87088,105.3994,79.39308,...,82.41758,90.49774,92.30769,89.61912,88.30455,92.30769,90.49774,93.24009,94.19152,96.15385


In [22]:
# Extract year from column names with year, and extract metadata information 
year_columns = rental_affordability.columns.str.extract(r'(\d{4})')[0]

# If the above pattern was matched, then it will be the year (e.g. 2012), but if it wasn't matched, then it returns NaN
pattern_columns = rental_affordability.columns[year_columns.notna()]
metadata_columns = rental_affordability.columns[year_columns.isna()]

In [23]:
rental_affordability_clean = pd.DataFrame()
unique_years = sorted(year_columns.dropna().unique())

for year in unique_years:
    # Select columns for the current year
    year_cols = rental_affordability.columns[year_columns == year]
    # Sum values across these columns
    rental_affordability_clean[year] = rental_affordability[year_cols].median(axis=1)

In [24]:
rental_years = []

for i in range(2011, 2022):
    rental_years.append(str(i))

In [25]:
rental_affordability_staging = pd.concat([rental_affordability[metadata_columns], rental_affordability_clean], axis=1)
rental_affordability_staging = rental_affordability_staging[rental_years]

In [26]:
# Since we want to look at all of greater Melbourne, we collapse the dataset and add all the scorings by year
year_sums = []

for year in rental_years:
    year_sums.append(rental_affordability_staging[year].median().round(2))

In [27]:
rental_affordability_final = pd.DataFrame()
rental_affordability_final['year'] = rental_years
rental_affordability_final['rai'] = year_sums

Thus, here are the 2 final datasets we will use for our analysis:

In [28]:
liveability_final

,year,percentage
0,2006,51.20
1,2011,52.91
2,2012,89.50
3,2013,9.49
4,2014,45.99
5,2015,47.51
6,2016,48.43
7,2017,47.85
8,2018,45.37


In [29]:
rental_affordability_final

,year,rai
0,2011,88.76
1,2012,89.91
2,2013,97.18
3,2014,97.68
4,2015,96.15
5,2016,100.59
6,2017,101.34
7,2018,100.83
8,2019,104.62
9,2020,133.01


## Data modelling

For these models, we also take the same approach as the other dataset analysis between building permits and population. 

Note that the overlap of years will be between 2011 ~ 2018 (inclusive of both ends), and the 2019~2021 data from the rental affordability index can be used to predict what the liveability percentage would've been if we had the data for those years. 

In [30]:
analysis_df = pd.merge(liveability_final, rental_affordability_final, how='inner')
analysis_df['year'] = analysis_df['year'].astype(int)
analysis_df

,year,percentage,rai
0,2011,52.91,88.76
1,2012,89.50,89.91
2,2013,9.49,97.18
3,2014,45.99,97.68
4,2015,47.51,96.15
5,2016,48.43,100.59
6,2017,47.85,101.34
7,2018,45.37,100.83


In [31]:
full_model = ols(
    formula='percentage ~ rai + year + rai*year',
    data=analysis_df
).fit()

print(full_model.summary())

                            OLS Regression Results                            
Dep. Variable:             percentage   R-squared:                       0.555
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     1.660
Date:                Mon, 20 May 2024   Prob (F-statistic):              0.311
Time:                        19:31:24   Log-Likelihood:                -32.132
No. Observations:                   8   AIC:                             72.26
Df Residuals:                       4   BIC:                             72.58
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.098e+04   2.12e+05     -0.194      0.8

/opt/homebrew/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


Appears that all coefficients are not significant? Remove interaction and see which variable between year and rai has more impact on the liveability percentage

In [32]:
nested_model_1 = ols(
    formula='percentage ~ rai + year',
    data=analysis_df
).fit()

print(nested_model_1.summary())

                            OLS Regression Results                            
Dep. Variable:             percentage   R-squared:                       0.554
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     3.101
Date:                Mon, 20 May 2024   Prob (F-statistic):              0.133
Time:                        19:31:24   Log-Likelihood:                -32.140
No. Observations:                   8   AIC:                             70.28
Df Residuals:                       5   BIC:                             70.52
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.288e+04   1.22e+04     -1.881      0.1

/opt/homebrew/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


In [33]:
nested_model_2 = ols(
    formula='percentage ~ year + rai',
    data=analysis_df
).fit()

print(nested_model_2.summary())

                            OLS Regression Results                            
Dep. Variable:             percentage   R-squared:                       0.554
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     3.101
Date:                Mon, 20 May 2024   Prob (F-statistic):              0.133
Time:                        19:31:24   Log-Likelihood:                -32.140
No. Observations:                   8   AIC:                             70.28
Df Residuals:                       5   BIC:                             70.52
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.288e+04   1.22e+04     -1.881      0.1

/opt/homebrew/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


In [34]:
nested_model_1.aic < nested_model_2.aic

False

From the above results, we can see that our final simple model should be nested_model_2. 

## Data visualisation

The following section contains code to visualise the data used in this scenario

In [35]:
analysis_df['Liveability Percentage'] = analysis_df['percentage']

In [36]:
# Calculate Correlation Matrix
corr_matrix = analysis_df[['percentage', 'rai']].corr()

# Heatmap Visualization of Correlation
fig1 = px.imshow(corr_matrix, text_auto=True, aspect="auto", labels=dict(color="Correlation Coefficient"),
                 x=['Liveability', 'Rental Affordability Index'],
                 y=['Liveability', 'Rental Affordability Index'],
                 title='Heatmap of Correlation Between Liveability and Rental Affordability')
fig1.update_xaxes(side="top")

In [37]:
# Trend Visualization of Rental Affordability Index Over the Years
fig2 = px.line(analysis_df, x='year', y='Liveability Percentage', markers=True, title='Liveability Score Over the Years', labels={'precentage': 'Liveability Percentage', 'year': 'Year'})
fig2.update_layout(plot_bgcolor='white', xaxis=dict(showline=True, showgrid=False, linecolor='black'), yaxis=dict(showgrid=True, gridcolor='grey'))

In [42]:
fig1.write_image()

TypeError: write_image() missing 1 required positional argument: 'file'

In [41]:
# Save plots into docs
plotly.offline.plot(fig1, filename='../docs/liveability_vs_rai_heatmap.html')
plotly.offline.plot(fig2, filename='../docs/liveability_trend.html')

FileNotFoundError: [Errno 2] No such file or directory: '../docs/liveability_vs_rai_heatmap.html'